<a href="https://colab.research.google.com/github/pds2021/a5-crown90/blob/assignment_5/Assignment_5_WebApp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Rock Paper Scissors Lizard Spock 

### Gameplan

1. The player has to start/allow the webcam
2. The player adjusts an [ROCK/ PAPER/ SCISSORS/ LIZARD/ SPOCK] by hand
3. The player has to push the [CAPTURE] button
4. The AI player chooses one too and presents its choice to the player
5. The winner will be promoted
6. End of game 

### Game Rules

1. scissors cuts paper
2. paper covers rock
3. rock crushes lizard
4. lizard poisons spock
5. spock smashes scissors
6. scissors decapitates lizard
7. lizard eats paper
8. paper disproves spock
9. spock vaporizes rock
10. rock crushes scissors

### Hands

* SCISSORS: 
> index finger + middle finger
* ROCK: 
> fist
* PAPER:
> flat hand
* LIZARD: 
> index finger up, middle fingers down, pinky up, thumb in: </br> the standard rock 'n' roll salute
* SPOCK: 
> vulcan salute

### Game code

In [13]:
# Install and import packages
!pip install -Uqq fastai  # upgrade fastai on colab
from fastai.vision.all import *
from fastai.vision.widgets import *
from IPython.display import Image
from IPython.display import display
from IPython.display import Javascript
from google.colab.output import eval_js
from base64 import b64decode
import random

In [14]:
# This class contains quick and dirty methods to manage the 
# Rock-Paper-Scissors-Lizard-Spock game
class Simple_RPSLS_Game(object):
  # Mapping of number to hand and its inverse
  Mapping_number_hand = {0:"rock", 1:"paper", 2:"scissors", 3:"lizard", 4:"spock"}
  Mapping_hand_number = {v: k for k, v in Mapping_number_hand.items()}
 
  # Matrix which describes the win-lose-combinations (with 0:rock, 1:paper, 2:scissors, 3:lizard, 4:spock, -1 tie)
  Matrix_win_lose = [[-1, 1, 0, 0, 4],[1, -1, 2, 3, 1], [0, 2, -1, 2, 4], [0, 3, 2, -1, 3], [4, 1, 4, 3, -1]]

  # Constructor
  def __init__(self, model_link='https://github.com/pds2021/a5-crown90/releases/download/v0.2/export.pkl'):
    download_url(model_link, 'rock-paper-scissors-lizard-spock.pkl')
    self.learn_inf = load_learner('rock-paper-scissors-lizard-spock.pkl', cpu=True)

  # Method to take a webcam picture
  def player_take_picture(self, filename='player_choice.jpg', quality=0.8):
    js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
  
      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      const capture = document.createElement('button');
      capture.textContent = '2. CAPTURE HAND';
      div.appendChild(capture);

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
    display(js)
    data = eval_js('takePhoto({})'.format(quality))
    binary = b64decode(data.split(',')[1])
    with open(filename, 'wb') as f:
      f.write(binary)
    return filename
 
  # This method takes an image filename and predicts the hand
  # The return-values contains the predicted hand as string, 
  # probability from the prediction and other probabilities as string 
  def detect_hand(self, filename='player_choice.jpg'):
     img = PILImage.create(filename)
     pred,pred_idx,probs = self.learn_inf.predict(img)
     pred_result = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'
     pred_result_all = f'Probabilities: {probs[0]:.04f} {probs[1]:.04f} {probs[2]:.04f} {probs[3]:.04f} {probs[4]:.04f}'
     return (pred, pred_result, pred_result_all)

  # This method plays a hand and decides who wins
  # The return-values contains a string with your hand,
  # a string with the computers hand and a string with the outcome
  def play_hand(self, player_hand_str):
    # Translate player hand into the mapped integer
    player_hand_int = self.Mapping_hand_number[player_hand_str] 
    # Get the computer move randomly
    computer_hand_int = random.randint(0, 4) 

    # Find the winner of the match
    winner = self.Matrix_win_lose[player_hand_int][computer_hand_int]
    message_you = "You choose " + self.Mapping_number_hand[player_hand_int].upper()
    message_com = "Computer chooses " + self.Mapping_number_hand[computer_hand_int].upper()

    if winner == player_hand_int:
      message_outcome = "YOU WIN!"
    elif winner == computer_hand_int:
      message_outcome = "YOU LOSE! COMPUTER WINS!"
    else:
      message_outcome = "TIE!" 
    
    return (message_you, message_com, message_outcome)

### Game

In [15]:
# Instanciate game object
game = Simple_RPSLS_Game()

In [16]:
# Define output widget for image
out_pl = widgets.Output()

# Define label widgets
lbl_stats = widgets.Label()
lbl_you = widgets.Label()
lbl_computer = widgets.Label()
lbl_outcome = widgets.Label()

In [17]:
# Add functionality for 'webcam'-button
# which will be invoked by on-click event
def on_click_webcam(change):
    filename = game.player_take_picture()
    img = PILImage.create(filename)

    out_pl.clear_output()
    with out_pl: display(img)
    
    pred, pred_result, pred_result_all = game.detect_hand()
    message_you, message_com, message_outcome = game.play_hand(pred)
    
    lbl_you.value = message_you
    lbl_computer.value = message_com
    lbl_outcome.value = message_outcome
    lbl_stats.value = f"{pred_result} ({pred_result_all})"

# Define button
btn_webcam = widgets.Button(description='1. START WEBCAM')
btn_webcam.on_click(on_click_webcam)

In [19]:
VBox([widgets.Label("Scissors - Rock - Paper - Lizard - Spock GAME"),
      btn_webcam, out_pl, lbl_stats, lbl_you, lbl_computer, lbl_outcome])

<IPython.core.display.Javascript object>